<a href="https://colab.research.google.com/github/NicolaGabriele/MastodonContentCompliance/blob/main/topic_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 77.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.8
    Uninstalling Cython-3.0.8:
      Successfully uninstalled Cython-3.0.8


In [2]:
import os
from bertopic import BERTopic
import json
RESULTS_HOME = '/kaggle/input/instance-json/results/results'
topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

2024-07-04 08:25:36.317421: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 08:25:36.317550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 08:25:36.440291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


topics.json:   0%|          | 0.00/10.0M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

topic_embeddings.safetensors:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

ctfidf_config.json:   0%|          | 0.00/4.66M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
'''topic, prob = topic_model.transform("This is an incredible movie!")
topic_model.topic_labels_[topic[0]]'''
from tqdm import tqdm

In [4]:
instances_file_path = '/kaggle/input/instance-json/instances.jsonl'

instances = []

with open(instances_file_path, 'r') as file:
    for line in file:
        instance_data = json.loads(line)
        instances.append(instance_data)

filtered_instances = [inst for inst in instances if inst["instance"] != "mastodon.social"]

valid_instances = []

for instance in filtered_instances:
    instance_file_path = os.path.join(RESULTS_HOME, f"{instance['instance']}.json")
    if os.path.isfile(instance_file_path):
        valid_instances.append(instance['instance'])
    if len(valid_instances) == 15:
        break

print(valid_instances)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/instance-json/instances.jsonl'

In [4]:
def process_batch(posts_batch):
    t, _ = topic_model.transform(posts_batch)
    return [topic_model.topic_labels_[i] for i in t]

In [8]:
aligned_count = 0
total_count = 1
#for instance_name in os.listdir(RESULTS_HOME):
instance = json.load(open(os.path.join(RESULTS_HOME, 'mastodon.social.json'), 'r'))
posts = [post['text'] for post in tqdm(instance['records'], desc='Pull post' ) if post['language'] == 'en']
aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
tags = []
for tagl in tqdm(aux, desc='Processing tags'):
    for tag in tagl:
        tags.append(tag['name'])
        
batch_size = 10000
batches = [posts[i:i + batch_size] for i in range(0, len(posts), batch_size)]

topic_path = os.path.join('/kaggle/working', 'fullgas_topics.txt')

all_topics = []
for batch in tqdm(batches, desc='Processing batches'):
    topics = process_batch(batch)
    all_topics.extend(topics)
    
    with open(topic_path, 'a') as f:
        for topic in topics:
            f.write(f'{topic}\n')
    
if len(tags) > 0:
    total_count += len(tags)
    for tag in tqdm(tags, desc='Checking tag alignment'):
        for topic in all_topics:
            if tag in topic:
                aligned_count += 1
                break
                
output_path = os.path.join('/kaggle/working', 'count_results.txt')
with open(output_path, 'w') as file:
    file.write(f'Aligned Count: {aligned_count}\n')
    file.write(f'Total Count: {total_count}\n')
    file.write(f'Alignment / Total : {aligned_count / total_count:.4f}\n')
                
print(aligned_count)
print(total_count)
print(aligned_count/total_count)

Exception ignored in: <function _xla_gc_callback at 0x7c01fc2b5a20>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 
Processing batches:   0%|          | 0/29 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/29 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [15]:
instance = json.load(open(os.path.join(RESULTS_HOME, 'mastodon.social.json'), 'r'))
print(len(tags))
print(len(tags2))

400000
414704


In [6]:
from tqdm import tqdm
c = 0
for instance_name in tqdm(os.listdir(RESULTS_HOME)):
    instance = json.load(open(os.path.join(RESULTS_HOME, instance_name), 'r'))
    posts = [post['text'] for post in instance['records'] if post['language'] == 'en']
    aux = [post['tags'] for post in instance['records'] if post['language'] == 'en']
    tags = []
    for tagl in aux:
        for tag in tagl:
            tags.append(tag['name'])
    if len(tags)>0:
        c+= len(posts)
    
print(c)
#Numero di post totali da analizzare 18387780

100%|██████████| 788/788 [12:02<00:00,  1.09it/s]

18387780


In [7]:
!rm -r /kaggle/working

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


rm: cannot remove '/kaggle/working': Device or resource busy
